# BERT - Out of the Box

In this notebook, we will test the performance of an out-of-the-box BERT model on CommonsenseQA. I follow the tutorial here: https://github.com/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb

I've implemented the Hugginface transformers library. 

I referred to the Commonsense QA repo and code to understand how the authors of this work establiahsed their baseline using BERT. This is the link to their code: https://github.com/jonathanherzig/commonsenseqa/blob/master/bert/run_commonsense_qa.py

From this repo (README): https://github.com/jonathanherzig/commonsenseqa

Their work is far more advanced and complicated than maybe what I want to do at this time. But I refer to their work to understand the set up.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import warnings

import json
from pandas.io.json import json_normalize
warnings.filterwarnings('ignore')


from transformers import BertTokenizer, TFBertModel, BertConfig

import tensorflow as tf
from tensorflow import keras

from datetime import datetime

import time 
configuration = BertConfig() 
from IPython.display import Image 

from tensorflow.keras.callbacks import TensorBoard
import pickle

runtype="tiny"

print("Run type:", runtype)
print(tf.__version__) 
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ",
      len(tf.config.experimental.list_physical_devices('CPU')))


Run type: tiny
2.3.0
Num GPUs Available:  0
Num CPUs Available:  1


## Import dataset

It's in the dataset folder.

In [2]:
def load_data(file):
    lines = []
    with open(file, 'rb') as json_file:
        for json_line in json_file:
            lines.append(json.loads(json_line))
        data = json_normalize(lines)
        data.columns = data.columns.map(lambda x: x.split(".")[-1])
    return data
# os.chdir('w266-commonsenseqa/BERT_oob)
train = load_data('../dataset/train_rand_split.jsonl')
dev = load_data('../dataset/dev_rand_split.jsonl')
train.head()

,answerKey,id,question_concept,choices,stem
0,A,075e483d21c29a511267ef62bedc0461,punishing,"[{'label': 'A', 'text': 'ignore'}, {'label': '...",The sanctions against the school were a punish...
1,B,61fe6e879ff18686d7552425a36344c8,people,"[{'label': 'A', 'text': 'race track'}, {'label...",Sammy wanted to go to where the people were. ...
2,A,4c1cb0e95b99f72d55c068ba0255c54d,choker,"[{'label': 'A', 'text': 'jewelry store'}, {'la...",To locate a choker not located in a jewelry bo...
3,D,02e821a3e53cb320790950aab4489e85,highway,"[{'label': 'A', 'text': 'united states'}, {'la...",Google Maps and other highway and street GPS s...
4,C,23505889b94e880c3e89cff4ba119860,fox,"[{'label': 'A', 'text': 'pretty flowers.'}, {'...","The fox walked from the city into the forest, ..."


## Steps

1. Import training examples
2. Process it
    - Format input into something BERT can work with, including `[CLS]` and `[SEP]`
    - We were thinking which label is correct: 
    - Tokenize 
    - Create an output layer using softmax. 
3. Train it
    - Specify how many layers of BERT to fine tune
    

# BERT base model (uncased)

From: https://huggingface.co/bert-base-uncased

> Pretrained model on English language using a masked language modeling (MLM) objective. It was introduced in this paper and first released in this repository. This model is uncased: it does not make a difference between english and English.
> 
> Disclaimer: The team releasing BERT did not write a model card for this model so this model card has been written by the Hugging Face team.

For each question, there are five answer choices. Only one of them is correct.

For BERT, the first thought was to have all five answers attached to each question, and the model would choose one of the five responses. This is how it's originally done in the CommonsenseQA paper.

```
[CLS] Question text here [SEP] Ans choice A [SEP] Ans choice B [SEP] Ans choice C [SEP] Ans choice D [SEP] Ans choice E [SEP]
```

It seems complicated, however, and requires a significant lift. So for now, let me try creating five question-answer pairs for each question. Like this:

```
[CLS] Question text here [SEP] Ans choice A [SEP]
[CLS] Question text here [SEP] Ans choice B [SEP]
[CLS] Question text here [SEP] Ans choice C [SEP]
[CLS] Question text here [SEP] Ans choice D [SEP]
[CLS] Question text here [SEP] Ans choice E [SEP]
```

Only one of the above 5 inputs will have a positive label for being the correct answer. The rest will have 0. The problem with this model is that we're evaluating each choice separately to see if it looks like a right answer at all. But I think it's important for the model to know how the answer choices compare to each other as well.


In [3]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
lab_order = {"A": 0, "B":1, "C":2, "D":3, "E":4}

class InputExample(object):
    """A single multiple choice question."""
    # This class is adapted from https://github.com/jonathanherzig/commonsenseqa/blob/master/bert/run_commonsense_qa.py

    def __init__(
            self,
            qid,
            question,
            answer,
            label):
        """Construct an instance."""
        self.qid = qid
        self.question = question  # e.g., 'The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?'
        self.answer = answer      # e.g., "ignore" if choice label is A 
        self.label = label        # e.g., If correct answer, 1. Otherwise 0. 
        
    def __str__(self):
        return "QUESTION: {}\nANSWER  : {}\nLABEL   : {}".format(self.question, self.answer, self.label)
    
    
def create_example(row, choice_num):
    qid = row.id
    
    # Question: Just take it from stem 
    question = row.stem
    
    # Answer choice 
    label = int(row["answerKey"] == choice_num)  # If the answer key is equal to the answer choice number, mark 1 
    answer = row["choices"][lab_order[choice_num]]["text"]         # actual ans text 
    
    return InputExample(qid, question, answer, label) 
    
    
def process_examples(data):
    examples = []
    labels = []
    questions = []
    anscands = []
    
    
    for index, row in data.iterrows(): 
        for letter in lab_order.keys():
            example = create_example(row, letter)
            examples.append(example)
            
            questions.append(example.question)
            anscands.append(example.answer)
            labels.append(example.label)
    labels = np.array(labels)
        
    encoded_example = tokenizer(questions, anscands, padding=True, truncation=True, return_tensors='tf')
    
    # Make the encoded example a dictionary of np arrays
    for key in encoded_example:
        encoded_example[key] = np.array(encoded_example[key])
        
    clean_encoded_example = [
        encoded_example["input_ids"],
        encoded_example["attention_mask"],
        encoded_example["token_type_ids"]
    ]
            
    return examples, clean_encoded_example, labels


Process dev and training data. We'll just do the first few to make a tiny dataset.

In [4]:
dev_eg, dev_encoded_eg, dev_labs = process_examples(dev[0:2])
train_eg, train_encoded_eg, train_labs = process_examples(train.iloc[0:2])

### Game plan

I'll be following the tutorial on: http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/, but for specifically commonsenseQA. Here are the steps from the tutorial.

1. Embed all sentences. Let's look at the output from the BERT model for our inputs. 
2. The tutorial says do a train/test split. We don't need this 'cause our data came separate.
3. Train the logistic regressio model using the training set. This is training on the output of BERT. I will need to create a FFNN to attach at the end of BERT. See what it comes back with. 
4. (Optional for now) For each question, pick the answer with the highest score. 
5. Then, evaluate against true answers. The evaluation metric will be % of questions with the correct answers out of all questions. 


#### Step 1: Embed all sentences. 

I've already formatted the CommonsenseQA inpts to be fed into the BERT model. Let's look at the output from the BERT model for our inputs. It should have a 768-long vector for each input token.

### Sunday morning retry 

In [5]:
NAME = 'oob_bert_classify_{runtype}_{ts}'.format(runtype=runtype, ts=int(time.time()))
tensorboard = TensorBoard(log_dir='models/logs/{}'.format(NAME))

def classification_model(max_len):
    """Implementation of classification model.
    Returns: model"""
    ## BERT encoder
    encoder = TFBertModel.from_pretrained("bert-base-uncased")
    ## QA Model
    input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="InputID")
    attention_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="AttentionMask")
    token_type_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="TokenTypeID")
    embedding = encoder(
        [input_ids, attention_mask, token_type_ids]
    )[0]
    # Feed inputs through the bert model, 
    # then take just the vector associated with first token [CLS]
    bert_cls_output = embedding[:,0]
    # These are the layers that come after Bert.
    dense = tf.keras.layers.Dense(256, activation='relu', name='dense')(bert_cls_output)
    # Output layer to predict correct answer. 
    # For the future, we may modify it to choose the max candidate answer of each question
    # for now, just predict from 0 to 1 whether this looks like a correct answer. 
    pred = tf.keras.layers.Dense(1, activation='sigmoid', name='correct')(dense)
    model = tf.keras.models.Model(inputs=[input_ids, token_type_ids, attention_mask],
                                  outputs=pred)
    model.compile(loss="binary_crossentropy", 
                  optimizer="adam",
                  metrics=["accuracy"])
    model.summary()
    return model


# Train

On training set

In [6]:
# train_eg, train_encoded_eg, train_labs 

max_length= len(train_encoded_eg[0])   # num max token 
BertClassifierModel = classification_model(max_len=max_length)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
InputID (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
AttentionMask (InputLayer)      [(None, 10)]         0                                            
__________________________________________________________________________________________________
TokenTypeID (InputLayer)        [(None, 10)]         0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 10, 768), (N 109482240   InputID[0][0]                    
                                                                 AttentionMask[0][0]   

In [ ]:
start = time.time()

BertClassifierModel.fit(
    [   train_encoded_eg[0],
        train_encoded_eg[1],
        train_encoded_eg[2]],
    train_labs, 
    epochs=3,
    # Insert validation data
    validation_data=(
        [dev_encoded_eg[0],
         dev_encoded_eg[1],
         dev_encoded_eg[2]
        ], dev_labs
    ),
    # Log the training info on tensorboard
    callbacks=[tensorboard])

end = time.time()
print("Execution duration in minutes:", (end - start)/60)

Epoch 1/3


In [ ]:
predictions = BertClassifierModel.predict(
    [dev_encoded_eg[0],
         dev_encoded_eg[1],
         dev_encoded_eg[2]
        ])

location="models/predictions/BertClassifierModel_{}_dev_predictions".format(runtype)
pickle_out = open(location, "wb")
pickle.dump(predictions, pickle_out)
pickle_out.close()

In [ ]:
location='models/{NAME}.h5'.format(NAME=NAME)

BertClassifierModel.save(location)